In [ ]:
#using Markdown; Markdown.parse_file("README.md")

In [ ]:
MODEL_PATH_KEY = :wNB;
const FLTYPE = Float32;
include("heavyloadings.jl");

In [ ]:
half_b = load(MODEL_FOLDER*"\\bunny\\half_bunny_unmod.ply");
half_b_rot = load(MODEL_FOLDER*"\\bunny\\half_bunny_rot1.ply");

In [ ]:
include("firsticp.jl")

In [ ]:
half_b_sc = scaleMesh(half_b,10);
half_b_rot_sc = scaleMesh(half_b_rot,10);

In [ ]:
IJuliaCell(vis)

In [ ]:
placeMeshInLife(vis,half_b_sc,"hbunny","darkred");
placeMeshInLife(vis,half_b_rot_sc,"rotbunny","olivedrab2")

## Old workflow
Every array is using `Point3f0`

```julia
#Workflow: (reading, reference)
#Input meshes/pointclouds
reference_pc = deepcopy(half_b_sc);
reading_pc = noisifyMesh(half_b_rot_sc);
#Arrays of Point3f0
reference_array = vertices(reference_pc);
reading_array = vertices(reading_pc);
#KD tree for the reference array
#kdtree_ref = KDTree(reference_array);

#Sampling random elements from the reference array
sampled_points_ref = randomSamplePoints(reference_array,0.5);
#Building KD tree for the sampled points
kdtree_sampled_ref = KDTree(sampled_points_ref);

#pairing the points in the two meshes
indexer,fl_traits = createKnnPairArray(reading_array,kdtree_sampled_ref,sampled_points_ref,true);
#rejecting the worst 20% based on the distance
rej_is,rej_fltrs = rejectWorstPercent(indexer,fl_traits,0.2);
```

## New workflow
Using Homogeneous vector instead of `Point3f0`
### Workflow lépései:
1. [x] reading és ref mesh-t homogén koordinátás SVector/MVector**(?)** tömbbé alakítani
1. [x] a teljes referencia tömbből kd-tree
### Iteráció
1. [x] reading tömbből sampling
1. [x] samplinged tömbhöz párkeresés a fából
1. ##### Worst `x`% rejection
  * [x] párok sorbarendezése távolság szerint
  * [ ] a sorbarendezett párok `x`%-nak kidobása
1. ##### 2.5sigma rejection
  * [ ] szórás számítása
  * [ ] 2.5 sigmánál nagyobbak kidobása
1. [ ] A\b összeállítása és megoldása
1. [ ] transzformáció a megoldással
1. [ ] transzformáció mentése vizualizációhoz
1. [ ] kilépési feltétel check

### Iteráció vége

11. [ ] place in MeshCat
12. [ ] animate in MeshCat


In [ ]:
#Workflow: (reading, reference)
#Input meshes/pointclouds
reference_pc = deepcopy(half_b_sc);
reading_pc = noisifyMesh(half_b_rot_sc);
#Arrays of SVector
reference_array = convert2HomogeneousArray(vertices(reference_pc),FLTYPE);
reading_array = convert2HomogeneousArray(vertices(reading_pc),FLTYPE);
#Sampling random elements from the reference and the reading array
sampled_points_ref = randomSamplePoints(reference_array,0.5);
sampled_points_red = randomSamplePoints(reading_array,0.5);
#Building KD tree for the sampled ref points and the whole ref array
kdtree_sampled_ref = KDTree(sampled_points_ref);
kdtree_ref = KDTree(reference_array);

In [ ]:
#pairing the points in the two meshes
indexer,fl_traits = createKnnPairArray(sampled_points_red,kdtree_ref);

In [ ]:
function sortIndandTraits(indexes,traits)
    s_it = sortperm(traits[:,2])

In [ ]:
iddi = 1:10;

In [ ]:
println(euclidean.(sampled_points_red[iddi],reference_array[indexer[iddi]]))

println(fl_traits[iddi,2])

In [ ]:
sorter_it = sortperm(fl_traits[:,2]);
sorter_sorter_it = sortperm(sorter_it);

In [ ]:
sorted_fl = fl_traits[sorter_it,:]; # sorted by distance

In [ ]:
sorted_indexer = indexer[sorter_it,:];

In [ ]:
iddi = 1:10

In [ ]:
#how to get the paired points
println(euclidean.(reference_array[indexer[iddi,1]],sampled_points_red[indexer[iddi,2]]))
println(fl_traits[iddi,2])
println(fl_traits[indexer[iddi,2],2])

In [ ]:
#how to get the paire points after sorting
println(euclidean.(reference_array[sorted_indexer[iddi],1],sampled_points_red[sorted_indexer[iddi,2]]))
println(sorted_fl[iddi,2])
println(sorted_fl[sorter_sorter_it[sorted_indexer[iddi,2]],2])

In [ ]:
#rejecting the worst 20% based on the distance
rej_id, rej_fltrs = rejectWorstPercent(sorted_indexer,sorted_fl,0.2);

In [ ]:
#how to get the paired point after rejection
println(euclidean.(reference_array[rej_id[iddi],1],sampled_points_red[rej_id[iddi,2]]))
println(rej_fltrs[iddi,2])

In [ ]:
argi = (collect(1:10),collect(11:20),collect(21:30));
x1 = collect(1:10);
x2 = collect(11:20);
x3 = collect(21:30);

In [ ]:
[size(argi[i],1) for i in 1:length(argi)]

In [ ]:
function testi(pr,tupi...)
    ret = ()
    
    for i in 1:length(tupi)
        ret = (ret...,tupi[i][1:5])
    end
    return ret
end
function testi(pr,arr)
    return arr[1:5]
end

In [ ]:
r1,r2,r3=testi(0.2,x1,x2,x3)

In [ ]:
r1=testi(0.2,x1)

In [ ]:
r3

In [ ]:
@inline function allequal_3(x)
    length(x) < 2 && return true
    e1 = x[1]
    i = 2
    @inbounds for i=2:length(x)
        x[i] == e1 || return false
    end
    return true
end

In [ ]:
allequal_3([size(argi[i],1) for i in 1:length(argi)])